In [ ]:
pip install mtcnn

     |████████████████████████████████| 2.3MB 8.9MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN
 
# extract a single face from a given photograph
def extract_face(filename, required_size=(160, 160)):
	image = Image.open(filename)
	image = image.convert('RGB')
	print(filename)
	pixels = asarray(image)
	detector = MTCNN()
	results = detector.detect_faces(pixels)
	x1, y1, width, height = results[0]['box']
	x1, y1 = abs(x1), abs(y1)
	x2, y2 = x1 + width, y1 + height
	face = pixels[y1:y2, x1:x2]
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array
 
# load images and extract faces for all images in a directory
def load_faces(directory):
	faces = list()
	for filename in listdir(directory):
		path = directory + filename
		face = extract_face(path)
		faces.append(face)
	return faces
 
# load a dataset 
def load_dataset(directory):
	X, y = list(), list()
	for subdir in listdir(directory):
		path = directory + subdir + '/'
		if not isdir(path):
			continue
		faces = load_faces(path)
		labels = [subdir for _ in range(len(faces))]	
		print('>loaded %d examples for class: %s' % (len(faces), subdir))
		X.extend(faces)
		y.extend(labels)
	return asarray(X), asarray(y)
 

trainX, trainy = load_dataset('/content/drive/My Drive/ColabNotebooks/dataset/train/')
print(trainX.shape, trainy.shape)

testX, testy = load_dataset('/content/drive/My Drive/ColabNotebooks/dataset/val/')
print(testX.shape, testy.shape)



/content/drive/My Drive/ColabNotebooks/dataset/train/YoshihideSuga/YoshihideSuga_0002.png
/content/drive/My Drive/ColabNotebooks/dataset/train/YoshihideSuga/YoshihideSuga_0001.png
/content/drive/My Drive/ColabNotebooks/dataset/train/YoshihideSuga/YoshihideSuga_0005.png
/content/drive/My Drive/ColabNotebooks/dataset/train/YoshihideSuga/YoshihideSuga_0006.png
/content/drive/My Drive/ColabNotebooks/dataset/train/YoshihideSuga/YoshihideSuga_0003.png
>loaded 5 examples for class: YoshihideSuga
/content/drive/My Drive/ColabNotebooks/dataset/train/ZuzanaCaputova/ZuzanaCaputova_0005.png
/content/drive/My Drive/ColabNotebooks/dataset/train/ZuzanaCaputova/ZuzanaCaputova_0006.png
/content/drive/My Drive/ColabNotebooks/dataset/train/ZuzanaCaputova/ZuzanaCaputova_0008.png
/content/drive/My Drive/ColabNotebooks/dataset/train/ZuzanaCaputova/ZuzanaCaputova_0004.png
/content/drive/My Drive/ColabNotebooks/dataset/train/ZuzanaCaputova/ZuzanaCaputova_0002.png
/content/drive/My Drive/ColabNotebooks/dataset

In [ ]:
savez_compressed('/content/drive/My Drive/ColabNotebooks/data.npz', trainX, trainy, testX, testy)

In [ ]:
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model
 
# get the face embedding for one face
def get_embedding(model, face_pixels):
	face_pixels = face_pixels.astype('float32')
	mean, std = face_pixels.mean(), face_pixels.std()
	face_pixels = (face_pixels - mean) / std
	samples = expand_dims(face_pixels, axis=0)
	yhat = model.predict(samples)
	return yhat[0]
 
# load the face dataset
data = load('/content/drive/My Drive/ColabNotebooks/data.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)
# load the facenet model
model = load_model('/content/drive/My Drive/ColabNotebooks/facenet_keras.h5')
print('Loaded Model')
# convert each face in the train set to an embedding
newTrainX = list()
for face_pixels in trainX:
	embedding = get_embedding(model, face_pixels)
	newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)
# convert each face in the test set to an embedding
newTestX = list()
for face_pixels in testX:
	embedding = get_embedding(model, face_pixels)
	newTestX.append(embedding)
newTestX = asarray(newTestX)
print(newTestX.shape)

Loaded:  (436, 160, 160, 3) (436,) (96, 160, 160, 3) (96,)
Loaded Model
(436, 128)
(96, 128)


In [ ]:
#model2 = load_model('/content/drive/My Drive/ColabNotebooks/facenet_keras.h5')
model2.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
________________________________________________________________________________

In [ ]:
savez_compressed('/content/drive/My Drive/ColabNotebooks/faces-embeddings.npz', newTrainX, trainy, newTestX, testy)

In [ ]:
from numpy import load
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
# load dataset
data = load('/content/drive/My Drive/ColabNotebooks/faces-embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
# normalize input vectors
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)
# fit model
model = SVC(kernel='linear', probability=True)
model.fit(trainX, trainy)
# predict
yhat_train = model.predict(trainX)
yhat_test = model.predict(testX)
# score
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(testy, yhat_test)
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))


Accuracy: train=99.541, test=89.583


In [ ]:

from random import choice
from numpy import load
from numpy import expand_dims
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from matplotlib import pyplot
# load faces
data = load('/content/drive/My Drive/ColabNotebooks/data.npz')
testX_faces = data['arr_2']
# load face embeddings
data = load('/content/drive/My Drive/ColabNotebooks/faces-embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
# normalize input vectors
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)
# fit model
model = SVC(kernel='linear', probability=True)
model.fit(trainX, trainy)

# test model on a random example from the test dataset
selection = choice([i for i in range(testX.shape[0])])
random_face_pixels = testX_faces[selection]
random_face_emb = testX[selection]
random_face_class = testy[selection]
random_face_name = out_encoder.inverse_transform([random_face_class])
# prediction for the face
samples = expand_dims(random_face_emb, axis=0)
yhat_class = model.predict(samples)

# get name
predict_names = out_encoder.inverse_transform(yhat_class)
print('Predicted: %s' % (predict_names[0]))
print('Expected: %s' % random_face_name[0])
# plot
pyplot.imshow(random_face_pixels)
title = '%s' % (predict_names[0])
pyplot.title(title)
pyplot.show()